In [51]:
import pandas as pd
from tqdm import tqdm
import os
import nltk
from nltk.corpus import stopwords

In [52]:
brexit_file = pd.read_csv('./brexit_full_updated4.csv')

politicians = []

count_zero = 0

politicians_party ={}
politicians_party2 ={}

for index, row in brexit_file.iterrows():
    full_name = row['full name']
    party = row['party']
    party2 = row['Party abbreviation']
    last_name = full_name.split(" ")[-1]
    first_name = full_name.split(" ")[0]
    short_name = last_name.lower() + '.' + first_name.lower()

    if party == 0:
        count_zero += 1
    if party in [0, 1]:
        politicians.append(short_name)
        politicians_party[short_name] = party
    politicians_party2[short_name] = party2

print(len(politicians))
politicians = list(set(politicians))
print(len(politicians))

594
594


In [62]:
stopW = stopwords.words('english')

def contains_european(x):
    y = x.lower()
    words = y.split()
    flag = 'e.u' in words or 'eu' in words or 'europeanunion' in words
    words = [word for word in words if word not in stopW]
    bigrams = list(nltk.bigrams(words))
    bigrams = [bigram[0] + '.' + bigram[1] for bigram in bigrams]
    
    flag = flag or 'european.union' in bigrams or 'europe.union' in bigrams
    
    return 'EU' if flag else 'non_EU'

def get_df(bbc_id):
    path = '../data/bbc/{}/{}/transcripts/'
    path1 = path.format(bbc_id, 2015)
    path2 = path.format(bbc_id, 2016)
    
    files = os.listdir(path1)
    df_list = []
    for _file in files:
        df_list.append(pd.read_csv(os.path.join(path1, _file)))
    df1 = pd.concat(df_list)
    
    files = os.listdir(path2)
    df_list = []
    for _file in files:
        df_list.append(pd.read_csv(os.path.join(path2, _file)))
        
    df2 = pd.concat(df_list)

    df1['Date'] = pd.to_datetime(df1['Date'])
    df1['month'] = df1['Date'].apply(lambda x: x.month)
    df1 = df1.loc[df1.month >= 6]
    df2['Date'] = pd.to_datetime(df2['Date'])
    df2['month'] = df2['Date'].apply(lambda x: x.month)
    df2 = df2.loc[df2.month <= 6]
    
    df = pd.concat([df1, df2])
    df = df.drop(['Unnamed: 0', 'month'], axis=1)
    rows = []
    df['year'] = df['Date'].apply(lambda x: x.year)
    df['month'] = df['Date'].apply(lambda x: x.month)
    df = df.dropna(subset=['Transcript'])
    df['eu'] = df['Transcript'].apply(lambda x: contains_european(x))
    df = df.loc[df['eu'] == 'non_EU']
    display(df.head())
    return df

In [63]:
df = get_df(57)

,Source,Date,Program Name,Time,Duration,Has Transcript,Transcript,year,month,eu
0,ITV+Website,2015-06-01,Migrants' desperate flight from one life in th...,NaN,NaN,True,"“Excuse me, what country am I in?” - if ever a...",2015,6,non_EU
1,ITV+Website,2015-06-01,Family calls for review of NHS funding after g...,NaN,NaN,True,A 12-year-old girl who has been denied potenti...,2015,6,non_EU
2,ITV+Website,2015-06-01,Crematorium apologises after failing to hand b...,NaN,NaN,True,A crematorium has apologised for causing distr...,2015,6,non_EU
3,ITV+Website,2015-06-01,Owner charged with murder over 2013 Dhaka fact...,NaN,NaN,True,The owner of a building that killed more than ...,2015,6,non_EU
4,ITV+Website,2015-06-01,Toddler placed for adoption due to level of ci...,NaN,NaN,True,A toddler has been taken from his parents and ...,2015,6,non_EU


# Count Functions

In [64]:
# count of times last names appears when first name also appears
def count1(text, last_name, first_name):
    words = text.split(" ")
    
    if first_name in words and last_name in words:
        return words.count(last_name)
    else:
        return 0

# count of articles where first name and last name both appears
def count2(text, last_name, first_name):
    words = text.split(" ")
    
    if first_name in words and last_name in words:
        return 1
    else:
        return 0

# count when last name and first name appears together
def count3(text, last_name, first_name):
    words = text.split(" ")
    
    n_words = len(words)
    
    c = 0
    for i in range(n_words-1):
        if words[i] == first_name and words[i+1] == last_name:
            c += 1

    return c

# count when last name and first name appears together
def count3_new(text, last_name, first_name):
    return text.count(first_name + ' ' + last_name) + text.count(first_name + last_name)

# count when last name and first name appears together
def count4(text, last_name, first_name):
    words = text.split(" ")
    
    n_words = len(words)
    
    for i in range(n_words-1):
        if words[i] == first_name and words[i+1] == last_name:
            return 1

    return 0

def count4_new(text, last_name, first_name):
    patt1 = first_name + ' ' + last_name
    patt2 = first_name + last_name
    
    if patt1 in text or patt2 in text:
        return 1
    
    return 0

In [65]:
bbc_ids = [57]
# print(politicians)
for bbc_id in bbc_ids:
    df = get_df(bbc_id)
#     df = df.loc[df['eu'] == 'non_EU']
    for p in tqdm(politicians):
        last_name, first_name = p.split('.')
        df[p] = df['Transcript'].apply(lambda x: count3_new(x.lower(), last_name, first_name))
            
    overall_count_df = None
    
    for p in politicians:
        count_df = df[['month', 'year', p]]
        count_df = count_df.groupby(['month', 'year'], as_index=False).sum()

        if overall_count_df is None:
            overall_count_df = count_df
        else:
            overall_count_df[p] = count_df[p]
    overall_count_df = overall_count_df.sort_values(by=['year', 'month']).reset_index(drop=True)
                      
    overall_count_df.to_csv('politicians_last_name_count_bbc_{}.csv'.format(bbc_id), index=False)
    rows = []

    for index, row in overall_count_df.iterrows():
        party_count = [0, 0, 0, 0]
        for p in politicians:
            if politicians_party[p] == 0:
                party_count[0] += row[p]
            else:
                party_count[1] += row[p]
                
            if politicians_party2[p] == 'Con':
                party_count[2] += row[p]
            else:
                party_count[3] += row[p]
                
        rows.append([row['month'], row['year'], party_count[0], party_count[1], party_count[2], party_count[3]])

    res_df = pd.DataFrame(rows, columns=['month', 'year', 'party=0', 'party=1', 'conservative', 'labour'])
    res_df = res_df.sort_values(by=['year', 'month']).reset_index(drop=True)
    res_df.to_csv('politicians_last_name_count_bbc_{}_by_party.csv'.format(bbc_id), index=False)

,Source,Date,Program Name,Time,Duration,Has Transcript,Transcript,year,month,eu
0,ITV+Website,2015-06-01,Migrants' desperate flight from one life in th...,NaN,NaN,True,"“Excuse me, what country am I in?” - if ever a...",2015,6,non_EU
1,ITV+Website,2015-06-01,Family calls for review of NHS funding after g...,NaN,NaN,True,A 12-year-old girl who has been denied potenti...,2015,6,non_EU
2,ITV+Website,2015-06-01,Crematorium apologises after failing to hand b...,NaN,NaN,True,A crematorium has apologised for causing distr...,2015,6,non_EU
3,ITV+Website,2015-06-01,Owner charged with murder over 2013 Dhaka fact...,NaN,NaN,True,The owner of a building that killed more than ...,2015,6,non_EU
4,ITV+Website,2015-06-01,Toddler placed for adoption due to level of ci...,NaN,NaN,True,A toddler has been taken from his parents and ...,2015,6,non_EU


100%|██████████| 594/594 [00:21<00:00, 27.36it/s]
